# Benchmark comparison of protein sequence preprocessing effect on learning task for Pfam family classification

Notebook below allows to play with parameters of the workflow and yield different results.
As a default state it is set with parameters described in the paper as well as used as a base for plots.

In [1]:
import os

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, interact_manual, fixed
import sys
from urllib import request
from scripts.data_preparation_support import data_preparation, split_to_train_test
from scripts.shorten_encoding import compress_protein_data_original, compress_protein_data_singletons, compress_protein_data_triplets, compress_protein_data_sum_of_triplets, compress_protein_data_sum_of_k_mers
from scripts.prepare_vectors import split_data_to_classes, prepare_biovec_model, load_biovec_model_with_classes
from scripts.prepare_input_stats import input_analysis
from scripts.model_scripts.decision_trees import decision_tree_func
from scripts.model_scripts.random_tree import random_tree_func
from scripts.model_scripts.mlp import mlp_func
from scripts.model_scripts.nearest_neighbours import nearest_neighbours_func
from scripts.model_scripts.deep_learning import deep_learning_func
from scripts.plotting_support import results_plot_benchmark, plot_sizes
from time import time

2022-06-26 11:54:31.615926: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-26 11:54:31.615959: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Dataset
In this project I provide already prepared - cleaned dataset used for the report, so download part below is commented out. However, if desired, parameters can be tweaked to obtain altered version.

Raw data for this project is easily accessible by the Swissprot part of the Uniprot Database.
Unfortunately, full file weights around 250Mb, so instead I provide link for download.

In case server can't handle the download through Python, it is still possiblle to download tab-separated file directly
from the website: [LINK](https://www.uniprot.org/uniprot/?query=reviewed%3Ayes&columns=id%2Cdatabase(Pfam)%2Corganism%2Csequence)

It suggested, although not neccessary, to put downloaded file into ./data/full directory for clarity and ease of use of the default values below.

In [4]:
# url="https://www.uniprot.org/uniprot/?query=reviewed:yes&format=tab&columns=id,database(Pfam),organism,sequence"
# request.urlretrieve(url, "./data/full/uniprot-reviewed_yes.tab")

In [5]:
def data_prepare_widget():
    org_w = widgets.BoundedIntText(value=2000, min=1, max=5000, step=1, description="Organisms")
    fam_w = widgets.BoundedIntText(value=10, min=1, max=200, step=1, description="Families")
    infile = widgets.Text(value="./data/full/uniprot-reviewed_yes.tab",  description="Input path")
    outfile = widgets.Text(value="./data/data_file.fasta", description="Output path")

    widget = interact_manual.options(manual_name="Prepare data")
    widget(data_preparation, n_org=org_w, n_fam=fam_w, file_path=infile, outfile_path=outfile)

In [6]:
data_prepare_widget()

interactive(children=(Text(value='./data/full/uniprot-reviewed_yes.tab', description='Input path'), Text(value…

At this point of code raw Swissprot data was filtered by the top number of organisms and picked proteins containing single domain from top biggest families.

However, before splitting our data to train and test, we want to first adress very simmilar sequences.
It can be done using CD-HIT package.

In [7]:
def run_cdhit(input_f="./data/to_cluster/data_file.fasta", output_f="./data/to_cluster/data_file_outttt.fasta", c="0.99"):
    c = str(c)
    print("Begin CD-HIT")
    os.system(f"./CD-HIT/cd-hit -i {input_f} -o {output_f} -c {c}")


def run_cdhit_widget():
    c = widgets.BoundedFloatText(value=0.99, min=0.7, max=1, description="Simmilarity")
    infile = widgets.Text(value="./data/data_file.fasta",  description="Input path")
    outfile = widgets.Text(value="./data/clustering/data_file_clustered.fasta", description="Output path")

    widget = interact_manual.options(manual_name="Prepare data")
    widget(run_cdhit, c=c, input_f=infile, output_f=outfile)

In [8]:
run_cdhit_widget()

interactive(children=(Text(value='./data/data_file.fasta', description='Input path'), Text(value='./data/clust…

Now our data is cleaned out and ready to process.

## Data processing

For this project data is processed in multiple ways.

 - Standard single-letter encoding
 - Conversion into numbers and using int-8 encoding
 - Conversion into 3-mers, encoding each 3-mer as a number and using int-16 encoding
 - Conversion into 3-mers, calculating count of the most popular triplets in sequences.
 - Conversion into k-mers, grouping with allowed edit distance and counting existance of fragments in group. (7-mers with edit distance 3)
 - Using Biovec vector encoder

First we need to split our data into training and test parts.
To avoid dominance of the biggest families, training data will contain the same number of sequences for each PFAM family.

### Warning!
Please analyse the histogram first before providing number of sequences per family to train.
Families with number of representatives lower than provided will be filtered out!

Sometimes it might be beneficial to filter out couple families with low coverage.

In [12]:
def histogram_widget():
    infile = widgets.Text(value="./data/clustering/data_file_clustered.fasta",  description="Input path")
    widget = interact_manual.options(manual_name="Prepare histogram")

    widget(input_analysis, input_file=infile)

In [13]:
histogram_widget()

interactive(children=(Text(value='./data/clustering/data_file_clustered.fasta', description='Input path'), But…

In [14]:
def split_to_train_test_widget():
    train_val = widgets.BoundedIntText(value=790, min=1, max=5000, step=1, description="N. to train")
    infile = widgets.Text(value="./data/clustering/data_file_clustered.fasta",  description="Input path")
    outfile = widgets.Text(value="./data/clean_dataset.pkl", description="Output path")

    widget = interact_manual.options(manual_name="Prepare data")
    new_val = widget(split_to_train_test, train_val=train_val, infile=infile, outfile=outfile)
    return new_val

In [15]:
train_val_global = split_to_train_test_widget()

interactive(children=(BoundedIntText(value=650, description='N. to train', max=5000, min=1), Text(value='./dat…

This process will save data into convenient pickle object. This way instead of keeping 4 separate files or trying to split our data in one file we can easily load a list prepared to use.

In [16]:
compress_protein_data_original(infile='./data/clean_dataset.pkl', outfile='./data/clean_dataset_original.pkl')

In [17]:
compress_protein_data_singletons(infile='./data/clean_dataset.pkl', outfile='./data/clean_dataset_singletons.pkl')

In [18]:
compress_protein_data_triplets(infile='./data/clean_dataset.pkl', outfile='./data/clean_dataset_triplets.pkl')

In [ ]:
compress_protein_data_sum_of_triplets(infile='./data/clean_dataset.pkl', outfile='./data/clean_dataset_sum_triplets.pkl')

## Shorten sum k-mer

Plugin below allows for customized summed k-mer encoding. Parameters:

- input, output files
- K-mer length - size of the moving, overalpping window
- Min. occurences - How many times certain k-mer must exist in all sequences to be taken into consideration (filtering out highly mutated fragments, picking "popularity" strength)
- Allowed edit distance - Fragments with edit distance equal or lower than provided to already found fragments will be united into one group. Example: with value 1 strings: AAAAA and AAAAB will be united while AAAAA and AAABB treated as separate groups.

In [ ]:
def shorten_kmer_widget():
    infile = widgets.Text(value="./data/clean_dataset.pkl",  description="Input path")
    outfile = widgets.Text(value="./data/clean_dataset_sum_k_mers.pkl", description="Output path")
    n_val = widgets.BoundedIntText(value=7, min=3, max=15, step=1, description="k-mer length")
    k_val = widgets.BoundedIntText(value=20, min=1, max=1000, step=1, description="min. occurences")
    edit_val = widgets.BoundedIntText(value=2, min=1, max=3, step=1, description="allowed edit distance")

    widget = interact_manual.options(manual_name="Prepare data")
    widget(compress_protein_data_sum_of_k_mers, n=n_val, k=k_val, edit=edit_val, infile=infile, outfile=outfile)

In [ ]:
shorten_kmer_widget()

## Biovec

Biovec model is built on top of the original implementation, thus data handling must be a bit different, to fit authors requirements.
Sequences must be split into class fasta files, and one combined with identical names.
Combined file will be used to generate model, wchich will be then saved.
This saved model, is next loaded again, but this time, we also provide family information.

After this procedure we are finally left with a data ready to split into training and test, perform learning and predictions.


### Warning !
This time widget is not provided, because of the ./data/vectors/combined_corpus.fasta file.
If Biovec recognizes this file it will skip creating a new model.
That's why paths here are fixed to ensure, old corpus file is deleted.

### Warning !
It might happen, that during loading process there will be information, that model did not train on certain triplets.
It is connected with fragments containing extended alphabet like "X" and are ignored.

In [20]:
split_data_to_classes(infile="./data/clean_dataset.pkl", output_folder="./data/vectors/class_folder",
                          output_combined_file="./data/vectors/combined.fasta")

In [21]:
prepare_biovec_model(infile="./data/vectors/combined.fasta", outfile="./data/vectors/ProtVec_model.model")

In [22]:
def load_biovec_widget():
    train_val = widgets.BoundedIntText(value=790, min=1, max=5000, step=1, description="N. to train")

    widget = interact_manual.options(manual_name="Prepare data")

    new_val = widget(load_biovec_model_with_classes, train_size=train_val, input_model=fixed("./data/vectors/ProtVec_model.model"), class_folder=fixed("./data/vectors/class_folder"), outfile=fixed('./data/clean_dataset_biovec.pkl'))

In [23]:
load_biovec_widget()

interactive(children=(BoundedIntText(value=650, description='N. to train', max=5000, min=1), Button(descriptio…

## Evaluation

Now, that we have all data prepared, we can evaluate them both in accuracy and runtime.

Several models will be created - please note, that not all of them are equally suitable for this kind of data - the point is in efficiency comparison.

- Decision trees
- Random trees
- Nearest Neighbours
- MLP
- Simple Machine Learning with Dense layers

In [3]:
all_times = []
all_accs = []

In [3]:
s1 = time()
acc1, refit1 = decision_tree_func("./data/clean_dataset_original.pkl")
t1 = time() - s1

s2 = time()
acc2, refit2 = decision_tree_func("./data/clean_dataset_singletons.pkl")
t2 = time() - s2

s3 = time()
acc3, refit3 = decision_tree_func("./data/clean_dataset_triplets.pkl")
t3 = time() - s3

s4 = time()
acc4, refit4 = decision_tree_func("./data/clean_dataset_sum_triplets.pkl")
t4 = time() - s4

s5 = time()
acc5, refit5 = decision_tree_func("./data/clean_dataset_sum_k_mers.pkl")
t5 = time() - s5

s6 = time()
acc6, refit6 = decision_tree_func("./data/clean_dataset_biovec.pkl")
t6 = time() - s6

t1 = round(t1, 2)
t2 = round(t2, 2)
t3 = round(t3, 2)
t4 = round(t4, 2)
t5 = round(t5, 2)
t6 = round(t6, 2)

all_times.append([t1, t2, t3, t4, t5, t6])
all_accs.append([acc1, acc2, acc3, acc4, acc5, acc6])

Fitting 2 folds for each of 10 candidates, totalling 20 fits
Best model:
- max_depth: 20
----- Model accuracy: 0.169
----- Refit runtime: 1.7387
Fitting 2 folds for each of 10 candidates, totalling 20 fits
Best model:
- max_depth: 40
----- Model accuracy: 0.701
----- Refit runtime: 1.8949
Fitting 2 folds for each of 10 candidates, totalling 20 fits
Best model:
- max_depth: 10
----- Model accuracy: 0.649
----- Refit runtime: 0.9368
Fitting 2 folds for each of 10 candidates, totalling 20 fits
Best model:
- max_depth: 90
----- Model accuracy: 0.829
----- Refit runtime: 3.8933
Fitting 2 folds for each of 10 candidates, totalling 20 fits
Best model:
- max_depth: 100
----- Model accuracy: 0.309
----- Refit runtime: 10.2369
Fitting 2 folds for each of 10 candidates, totalling 20 fits
Best model:
- max_depth: 70
----- Model accuracy: 0.713
----- Refit runtime: 0.0496


In [4]:
print(f"Statistic | Original | Singletons | Triplets | Sum Triplets | Sum K-mers | Biovec")
print(f"----------|----------|------------|----------|--------------|------------|--------")
print(f"Time      | {t1} \t | {t2} \t | {t3} \t | {t4} \t | {t5} \t | {t6} \t")
print(f"Accuracy  | {acc1} \t | {acc2} \t | {acc3} \t | {acc4} \t | {acc5} \t | {acc6} \t")

Statistic | Original | Singletons | Triplets | Sum Triplets | Sum K-mers | Biovec
----------|----------|------------|----------|--------------|------------|--------
Time      | 1.7387 	 | 1.8949 	 | 0.9368 	 | 3.8933 	 | 10.2369 	 | 0.0496 	
Accuracy  | 0.1686 	 | 0.701 	 | 0.6486 	 | 0.8286 	 | 0.3086 	 | 0.7126 	


In [5]:
s1 = time()
acc1, refit1 = random_tree_func("./data/clean_dataset_original.pkl")
t1 = time() - s1

s2 = time()
acc2, refit2 = random_tree_func("./data/clean_dataset_singletons.pkl")
t2 = time() - s2

s3 = time()
acc3, refit3 = random_tree_func("./data/clean_dataset_triplets.pkl")
t3 = time() - s3

s4 = time()
acc4, refit4 = random_tree_func("./data/clean_dataset_sum_triplets.pkl")
t4 = time() - s4

s5 = time()
acc5, refit5 = random_tree_func("./data/clean_dataset_sum_k_mers.pkl")
t5 = time() - s5

s6 = time()
acc6, refit6 = random_tree_func("./data/clean_dataset_biovec.pkl")
t6 = time() - s6

t1 = round(t1, 2)
t2 = round(t2, 2)
t3 = round(t3, 2)
t4 = round(t4, 2)
t5 = round(t5, 2)
t6 = round(t6, 2)

all_times.append([t1, t2, t3, t4, t5, t6])
all_accs.append([acc1, acc2, acc3, acc4, acc5, acc6])

Fitting 2 folds for each of 15 candidates, totalling 30 fits
Best model:
- random_state: 1234
- max_depth: 20
- max_features: 20
----- Model accuracy: 0.419
----- Refit runtime: 2.6424
Fitting 2 folds for each of 15 candidates, totalling 30 fits
Best model:
- random_state: 1234
- max_depth: 30
- max_features: 20
----- Model accuracy: 0.802
----- Refit runtime: 2.5483
Fitting 2 folds for each of 15 candidates, totalling 30 fits
[CV 1/2] END ......................max_depth=10;, score=0.632 total time=   1.2s
[CV 1/2] END ......................max_depth=20;, score=0.647 total time=   1.0s
[CV 1/2] END ......................max_depth=30;, score=0.649 total time=   1.0s
[CV 1/2] END ......................max_depth=40;, score=0.643 total time=   0.9s
[CV 1/2] END ......................max_depth=50;, score=0.640 total time=   1.1s
[CV 1/2] END ......................max_depth=60;, score=0.635 total time=   1.2s
[CV 1/2] END ......................max_depth=70;, score=0.637 total time=   1.1s
[C

/home/exsto/anaconda3/envs/benchmark_project_env/lib/python3.8/site-packages/sklearn/model_selection/_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Best model:
- random_state: 1234
- max_depth: 10
- max_features: 15
----- Model accuracy: 0.884
----- Refit runtime: 0.5977


In [6]:
print(f"Statistic | Original | Singletons | Triplets | Sum Triplets | Sum K-mers | Biovec")
print(f"----------|----------|------------|----------|--------------|------------|--------")
print(f"Time      | {t1} \t | {t2} \t | {t3} \t | {t4} \t | {t5} \t | {t6} \t")
print(f"Accuracy  | {acc1} \t | {acc2} \t | {acc3} \t | {acc4} \t | {acc5} \t | {acc6} \t")

Statistic | Original | Singletons | Triplets | Sum Triplets | Sum K-mers | Biovec
----------|----------|------------|----------|--------------|------------|--------
Time      | 2.6424 	 | 2.5483 	 | 3.4068 	 | 2.9265 	 | 4.0896 	 | 0.5977 	
Accuracy  | 0.419 	 | 0.8019 	 | 0.7876 	 | 0.9781 	 | 0.3419 	 | 0.8839 	


In [4]:
s1 = time()
acc1, refit1 = nearest_neighbours_func("./data/clean_dataset_original.pkl")
t1 = time() - s1

s2 = time()
acc2, refit2 = nearest_neighbours_func("./data/clean_dataset_singletons.pkl")
t2 = time() - s2

s3 = time()
acc3, refit3 = nearest_neighbours_func("./data/clean_dataset_triplets.pkl")
t3 = time() - s3

s4 = time()
acc4, refit4 = nearest_neighbours_func("./data/clean_dataset_sum_triplets.pkl")
t4 = time() - s4

s5 = time()
acc5, refit5 = nearest_neighbours_func("./data/clean_dataset_sum_k_mers.pkl")
t5 = time() - s5

s6 = time()
acc6, refit6 = nearest_neighbours_func("./data/clean_dataset_biovec.pkl")
t6 = time() - s6

t1 = round(t1, 2)
t2 = round(t2, 2)
t3 = round(t3, 2)
t4 = round(t4, 2)
t5 = round(t5, 2)
t6 = round(t6, 2)

all_times.append([t1, t2, t3, t4, t5, t6])
all_accs.append([acc1, acc2, acc3, acc4, acc5, acc6])

Best model:
- weights: distance
- algorithm: auto
----- Model accuracy: 0.331
Best model:
- weights: distance
- algorithm: auto
----- Model accuracy: 0.746
Best model:
- weights: distance
- algorithm: auto
----- Model accuracy: 0.73
Best model:
- weights: distance
- algorithm: auto
----- Model accuracy: 0.719
Best model:
- weights: distance
- algorithm: auto
----- Model accuracy: 0.301


/home/exsto/anaconda3/envs/benchmark_project_env/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Best model:
- weights: uniform
- algorithm: auto
----- Model accuracy: 0.871


In [5]:
print(f"Statistic | Original | Singletons | Triplets | Sum Triplets | Sum K-mers | Biovec")
print(f"----------|----------|------------|----------|--------------|------------|--------")
print(f"Time      | {t1} \t | {t2} \t | {t3} \t | {t4} \t | {t5} \t | {t6} \t")
print(f"Accuracy  | {acc1} \t | {acc2} \t | {acc3} \t | {acc4} \t | {acc5} \t | {acc6} \t")

Statistic | Original | Singletons | Triplets | Sum Triplets | Sum K-mers | Biovec
----------|----------|------------|----------|--------------|------------|--------
Time      | 9.32 	 | 5.77 	 | 2.82 	 | 8.45 	 | 10.12 	 | 0.77 	
Accuracy  | 0.3314 	 | 0.7457 	 | 0.7305 	 | 0.719 	 | 0.301 	 | 0.8712 	


In [ ]:
s1 = time()
acc1, refit1 = mlp_func("./data/clean_dataset_original.pkl")
t1 = time() - s1

s2 = time()
acc2, refit2 = mlp_func("./data/clean_dataset_singletons.pkl")
t2 = time() - s2

s3 = time()
acc3, refit3 = mlp_func("./data/clean_dataset_triplets.pkl")
t3 = time() - s3

s4 = time()
acc4, refit4 = mlp_func("./data/clean_dataset_sum_triplets.pkl")
t4 = time() - s4

s5 = time()
acc5, refit5 = mlp_func("./data/clean_dataset_sum_k_mers.pkl")
t5 = time() - s5

s6 = time()
acc6, refit6 = mlp_func("./data/clean_dataset_biovec.pkl")
t6 = time() - s6

t1 = round(t1, 2)
t2 = round(t2, 2)
t3 = round(t3, 2)
t4 = round(t4, 2)
t5 = round(t5, 2)
t6 = round(t6, 2)

all_times.append([t1, t2, t3, t4, t5, t6])
all_accs.append([acc1, acc2, acc3, acc4, acc5, acc6])

Fitting 2 folds for each of 3 candidates, totalling 6 fits


/home/exsto/anaconda3/envs/benchmark_project_env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  warnings.warn(


Best model:
- hidden_layer_sizes: 128
- activation: relu
- solver: adam
----- Model accuracy: 0.253
----- Refit runtime: 86.0004
Fitting 2 folds for each of 3 candidates, totalling 6 fits
Best model:
- hidden_layer_sizes: 128
- activation: relu
- solver: adam
----- Model accuracy: 0.779
----- Refit runtime: 49.0562
Fitting 2 folds for each of 3 candidates, totalling 6 fits
Best model:
- hidden_layer_sizes: 128
- activation: relu
- solver: adam
----- Model accuracy: 0.385
----- Refit runtime: 18.7105
Fitting 2 folds for each of 3 candidates, totalling 6 fits
Best model:
- hidden_layer_sizes: 128
- activation: relu
- solver: adam
----- Model accuracy: 0.993
----- Refit runtime: 18.5641
Fitting 2 folds for each of 3 candidates, totalling 6 fits


In [ ]:
print(f"Statistic | Original | Singletons | Triplets | Sum Triplets | Sum K-mers | Biovec")
print(f"----------|----------|------------|----------|--------------|------------|--------")
print(f"Time      | {t1} \t | {t2} \t | {t3} \t | {t4} \t | {t5} \t | {t6} \t")
print(f"Accuracy  | {acc1} \t | {acc2} \t | {acc3} \t | {acc4} \t | {acc5} \t | {acc6} \t")

In [ ]:
s1 = time()
acc1, refit1 = deep_learning_func("./data/clean_dataset_original.pkl")
t1 = time() - s1

s2 = time()
acc2, refit2 = deep_learning_func("./data/clean_dataset_singletons.pkl")
t2 = time() - s2

s3 = time()
acc3, refit3 = deep_learning_func("./data/clean_dataset_triplets.pkl")
t3 = time() - s3

s4 = time()
acc4, refit4 = deep_learning_func("./data/clean_dataset_sum_triplets.pkl")
t4 = time() - s4

s5 = time()
acc5, refit5 = deep_learning_func("./data/clean_dataset_sum_k_mers.pkl")
t5 = time() - s5

s6 = time()
acc6, refit6 = deep_learning_func("./data/clean_dataset_biovec.pkl")
t6 = time() - s6

t1 = round(t1, 2)
t2 = round(t2, 2)
t3 = round(t3, 2)
t4 = round(t4, 2)
t5 = round(t5, 2)
t6 = round(t6, 2)

all_times.append([t1, t2, t3, t4, t5, t6])
all_accs.append([acc1, acc2, acc3, acc4, acc5, acc6])

In [ ]:
print(f"Statistic | Original | Singletons | Triplets | Sum Triplets | Sum K-mers | Biovec")
print(f"----------|----------|------------|----------|--------------|------------|--------")
print(f"Time      | {t1} \t | {t2} \t | {t3} \t | {t4} \t | {t5} \t | {t6} \t")
print(f"Accuracy  | {acc1} \t | {acc2} \t | {acc3} \t | {acc4} \t | {acc5} \t | {acc6} \t")

In [ ]:
names = ["Original", "Singletons", "Triplets", "Sum Triplets", "Sum K-mers", "Biovec"]
tests = ["Decision trees", "Random trees", "Nearest neighbours", "MLP", "Machine Learning"]
print(all_times)
print(all_accs)
results_plot_benchmark(names, tests, all_times, all_accs)

In [ ]:
files = ["./data/clean_dataset_original.pkl",
         "./data/clean_dataset_singletons.pkl",
         "./data/clean_dataset_triplets.pkl",
         "./data/clean_dataset_sum_triplets.pkl",
         "./data/clean_dataset_sum_k_mers.pkl",
         "./data/clean_dataset_biovec.pkl"]

plot_sizes(files, names, "./presentation/images/sizes.png")